# SQL Chat GPT
In this notebook, we will learn how to use a feature of langchain to turn Chat GPT into an agent knowledgeable about our database.

For Chat GPT, context is everything. Therefore, if GPT has our database as context and reference, we can ask it questions related to the data.

Langchain is a Python package with various useful integrations to use with Chat GPT. 

To access the data from our database, we need to provide GPT with a .db file containing the database information. We will use 

Import the necessary langchain libraries for the project.



In [40]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain import LLMChain, PromptTemplate
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

Read the CSV we are going to use as our Database.

In [15]:
players = pd.read_csv("FBREF_BigPlayers_2324.csv", delimiter=';')

Store our OpenAI key as an environment variable and establish our connection to the OpenAI API.

In [10]:
model_name = "gpt-3.5-turbo"
llm = OpenAI(temperature=0, verbose=True, model=model_name)

Define the prompt template

In [17]:
template = """
Given the following data:
{data}

You are an expert football data analyst. Answer the following question: {question}
"""

prompt = PromptTemplate(input_variables=["data", "question"], template=template)

In [41]:
# agent = create_csv_agent(
#     OpenAI(temperature=0, model='gpt-4'),
#     "FBREF_BigPlayers_2324.csv",
#     pandas_kwargs={"delimiter":";"},
#     verbose=True,
#     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
# )

agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    "FBREF_BigPlayers_2324.csv",
    pandas_kwargs={"delimiter":";"},
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [43]:
response = agent.run("Can you tell me the players with the most goals for each league?")
#print(response)
#print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('Competition')['Gls'].idxmax().apply(lambda x: df.loc[x])"}`
responded: To find the players with the most goals for each league, we need to group the data by the "Competition" column and then find the player with the highest number of goals in each group. Let me do that for you.

NameError: name 'df' is not defined
Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndata = {'Player': ['Abner', 'Paul Akouokou', 'Sergi Altimira', 'Ezequiel Ávila', 'Cédric Bakambu'],\n        'Competition': ['La Liga', 'La Liga', 'La Liga', 'La Liga', 'La Liga'],\n        'Gls': [0, 0, 1, 1, 0]}\n\ndf = pd.DataFrame(data)\n\ndf.groupby('Competition')['Gls'].idxmax().apply(lambda x: df.loc[x])"}`
responded: It seems I forgot to load the dataframe. Let me load the dataframe and then find the players with the most goals for each league.

NameError: name 'df' is not defined
Invoking: `python_repl_a

Load our database into a variable using the functions provided by langchain.

In [17]:
database = SQLDatabase.from_uri("sqlite:///englishfootball.db")

Create the chain that connects our database to Chat GPT.

In [18]:
db_chain = SQLDatabaseChain.from_llm(client_gpt, database, verbose=True)

From now on, we can use the `db_chain.run` method to communicate with our database.

In [19]:
db_chain.run("Who is the top scorer in the league?")



> Entering new SQLDatabaseChain chain...
Who is the top scorer in the league?
SQLQuery:SELECT appearances.team_id, teams.team_name, SUM(appearances.goals_for) AS total_goals
FROM appearances
INNER JOIN teams ON appearances.team_id = teams.team_id
GROUP BY appearances.team_id
ORDER BY total_goals DESC
LIMIT 1;
SQLResult: [('T-022', 'Manchester United', 8141)]
Answer:The top scorer in the league is Manchester United with 8141 goals.
> Finished chain.


'The top scorer in the league is Manchester United with 8141 goals.'

In [ ]:
db_chain.run("""Give me the 3 teams with the better defense """)



> Entering new SQLDatabaseChain chain...
Give me the 3 teams with the better defense 
SQLQuery:SELECT teams.team_name, standings.goals_against 
FROM teams 
INNER JOIN standings ON teams.team_id = standings.team_id 
ORDER BY standings.goals_against ASC 
LIMIT 3
SQLResult: [('Wigan Borough', 0), ('Aldershot', 0), ('Maidstone United', 0)]
Answer:Wigan Borough, Aldershot, Maidstone United
> Finished chain.


'Wigan Borough, Aldershot, Maidstone United'

In [ ]:
db_chain.run("""Why do Wigan Borough, Aldershot, and Maidstone United have the best defense?""")



> Entering new SQLDatabaseChain chain...
Why do Wigan Borough, Aldershot, and Maidstone United have the best defense?
SQLQuery:SELECT teams.team_name, standings.goals_against
FROM teams
INNER JOIN standings ON teams.team_id = standings.team_id
WHERE teams.team_name IN ('Wigan Borough', 'Aldershot', 'Maidstone United')
ORDER BY standings.goals_against ASC
LIMIT 5
SQLResult: [('Wigan Borough', 0), ('Aldershot', 0), ('Maidstone United', 0), ('Aldershot', 38), ('Wigan Borough', 39)]
Answer:Wigan Borough, Aldershot, and Maidstone United have the best defense because they have the lowest number of goals against in the standings table.
> Finished chain.


'Wigan Borough, Aldershot, and Maidstone United have the best defense because they have the lowest number of goals against in the standings table.'